In [1]:
import json
import pandas as pd # type: ignore
import random

In [2]:
from datasets import load_dataset
import pandas as pd
train_dataset = load_dataset("Tevatron/wikipedia-nq", split='dev')

Generating train split:   0%|          | 0/58622 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/6489 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3610 [00:00<?, ? examples/s]

In [3]:
ok_df = train_dataset.to_pandas()
ok_df.head()

,query_id,query,answers,positive_passages,negative_passages
0,0,who sings does he love me with reba,[Linda Davis],"[{'docid': '11828866', 'text': 'Does He Love Y...","[{'docid': '14525568', 'text': 'song. Accordin..."
1,1,where do the great lakes meet the ocean,[the Saint Lawrence River],"[{'docid': '151960', 'text': 'Great Lakes The ...","[{'docid': '8204739', 'text': 'open lakes are ..."
2,2,when does the new my hero academia movie come out,"[July 5 , 2018]","[{'docid': '20766125', 'text': 'would be joini...","[{'docid': '20766129', 'text': 'The ""Plus Ultr..."
3,3,who was the creator of victoria 's secret,[Roy Raymond],"[{'docid': '13258931', 'text': 'the show."" Vic...","[{'docid': '16623292', 'text': 'The Time of Ou..."
4,4,when did wesley leave last of the summer wine,[2002],"[{'docid': '4441861', 'text': 'made his debut ...","[{'docid': '808428', 'text': 'to Barry (Mike G..."


In [4]:
test_data = []
test_doc_data = []

In [5]:
df = ok_df[['query','positive_passages']]
df.head()

,query,positive_passages
0,who sings does he love me with reba,"[{'docid': '11828866', 'text': 'Does He Love Y..."
1,where do the great lakes meet the ocean,"[{'docid': '151960', 'text': 'Great Lakes The ..."
2,when does the new my hero academia movie come out,"[{'docid': '20766125', 'text': 'would be joini..."
3,who was the creator of victoria 's secret,"[{'docid': '13258931', 'text': 'the show."" Vic..."
4,when did wesley leave last of the summer wine,"[{'docid': '4441861', 'text': 'made his debut ..."


In [6]:
from collections import defaultdict
# Step 1: Get all unique docids and texts across all queries
all_docs = {}  # Dictionary to store docid -> text
for passages in df['positive_passages']:
    for passage in passages:
        all_docs[passage['docid']] = passage['text']

# Step 2: Create dictionaries to store posqueries and negqueries for each docid
doc_pos_queries = defaultdict(list)
doc_neg_queries = defaultdict(list)

# Step 3: Process each row in the original DataFrame
for idx, row in df.iterrows():
    query = row['query']
    pos_docids = set()

    # Add positive queries to corresponding docids
    for passage in row['positive_passages']:
        docid = passage['docid']
        doc_pos_queries[docid].append(query)
        pos_docids.add(docid)

    # Any docid not in pos_docids is negative for this query
    for docid in all_docs:
        if docid not in pos_docids:
            doc_neg_queries[docid].append(query)

# Step 4: Collect results into a new DataFrame
all_docids = set(doc_pos_queries.keys()).union(doc_neg_queries.keys())
new_data = {
    'docid': [],
    'text': [],
    'posqueries': [],
    'negqueries': []
}

for docid in all_docids:
    new_data['docid'].append(docid)
    new_data['text'].append(all_docs.get(docid, ""))  # Add the text for the document
    new_data['posqueries'].append(doc_pos_queries.get(docid, []))
    new_data['negqueries'].append(doc_neg_queries.get(docid, []))

# Step 5: Create the new DataFrame and filter out rows where either list is empty
new_df = pd.DataFrame(new_data)
new_df = new_df[(new_df['posqueries'].apply(len) > 0) & (new_df['negqueries'].apply(len) > 0)]

new_df.head()

,docid,text,posqueries,negqueries
0,906201,"with the other three ""Major or Grand Slam"" eve...",[who won the men 's wimbledon championship 2016],"[who sings does he love me with reba, where do..."
1,4843523,Michael Rapaport Michael David Rapaport (born ...,[who played phoebe 's boyfriend gary on friends],"[who sings does he love me with reba, where do..."
2,13515445,"the Sikh Empire was Sikh (78%), Hindu (12%), M...",[leader of sikh empire who became maharajah of...,"[who sings does he love me with reba, where do..."
3,1613707,efficient (lack of residue). The goal is to fi...,[what is the difference between dysphagia and ...,"[who sings does he love me with reba, where do..."
4,12123558,University of Central Florida College of Engin...,[where is the university of central florida lo...,"[who sings does he love me with reba, where do..."


In [7]:
len(new_df)

45158

In [8]:
# Iterate over the rows of the DataFrame
for idx, row in new_df.iterrows():
    doc_text = row['text']  # Get the document text
    posqueries = row['posqueries']  # List of positive queries
    negqueries = row['negqueries']  # List of negative queries
    
    # Shuffle posqueries and negqueries
    random.shuffle(posqueries)
    random.shuffle(negqueries)
    
    # Append the document and queries to the list
    test_doc_data.append({
        'doc': doc_text,
        'pos_que': posqueries[:min(5, len(posqueries))],  # Correct slicing for positive queries
        'neg_que': negqueries[:20 - min(5, len(posqueries))]  # Correct slicing for negative queries
    })

In [9]:
len(test_doc_data)

45158

In [10]:

import torch
from torch import nn
import torch.nn.functional as F
from transformers import BertTokenizer, BertModel, BertForMaskedLM

import torch
from torch import nn

class ContrastiveLoss(nn.Module):
    """
        Given a list of scores s1,s2,..sn, calculates -log(e^s1/(e^s1+e^s2+...+e^sn))
    """
    def __init__(self):
        super(ContrastiveLoss, self).__init__()

    def forward(self, scores):
        scaled_scores = scores / 1.0
        max_score = torch.max(scaled_scores)
        stable_scaled_scores = scaled_scores - max_score
        log_sum_exp = max_score + torch.log(torch.sum(torch.exp(stable_scaled_scores)))
        loss = log_sum_exp - scaled_scores[0]

        return loss
    
    
class BertClsFFN(nn.Module):
    """
        A small feed forward network on top of CLS embedding, to get a score
    """
    def __init__(self):
        super(BertClsFFN, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.ffn = nn.Sequential(
            nn.Linear(768, 32),
            nn.ReLU(),
            nn.LayerNorm(32),
            nn.Linear(32, 8),
            nn.ReLU(),
            nn.LayerNorm(8),
            nn.Linear(8, 1),

        )
        self.freeze_bert()

    
    def freeze_bert(self):
        self.bert.embeddings.requires_grad_(False)
        for param in self.bert.encoder.layer[:11].parameters():
            param.requires_grad = False

    def forward(self, input_tokens):
        sentence_embed = self.bert(**input_tokens).pooler_output
        scores = self.ffn(sentence_embed).reshape(-1)
        return scores



class BertLogitScorer(nn.Module):
    """
        If the input format is [CLS] `sent1` [SEP] `sent2` [SEP], we sum the log_probs of tokens of `sent2` to get a representation of a score
    """
    def __init__(self):
        super(BertLogitScorer, self).__init__()
        self.bert = BertForMaskedLM.from_pretrained('bert-base-uncased')
        self.freeze_bert()

    def freeze_bert(self):
        self.bert.bert.embeddings.requires_grad_(False)
        for param in self.bert.bert.encoder.layer[:10].parameters():
            param.requires_grad = False

    def forward(self, input_tokens):
        input_ids = input_tokens['input_ids']
        batch_size, seq_length = input_ids.shape
        logits = self.bert(**input_tokens).logits
        log_probs = F.log_softmax(logits, dim=-1)

        sums = torch.zeros(batch_size, device=logits.device)

        for i in range(batch_size):
            sep_indices = (input_ids[i] == 102).nonzero(as_tuple=True)[0]
            idx1, idx2 = sep_indices[0].item(), sep_indices[1].item()
            token_ids_in_range = input_ids[i, idx1 + 1:idx2]
            log_probs_in_range = log_probs[i, idx1 + 1:idx2]
            gathered_log_probs = torch.gather(log_probs_in_range, dim=1, index=token_ids_in_range.unsqueeze(-1)).squeeze(-1)
            
            sums[i] = torch.sum(gathered_log_probs)

        return sums


In [11]:
class DocLH_CLS(nn.Module):
    def __init__(self, model_path=None):
        super(DocLH_CLS,self).__init__()
        self.bert_scorer = BertClsFFN()
        if model_path is not None:
            self.bert_scorer.load_state_dict(torch.load(model_path))
        self.bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    
    def forward(self, d):
        texts = []
        for pd in d['pos_doc']:
            text = d['query'] + ' [SEP] ' + pd
            texts.append(text)
        for nd in d['neg_doc']:
            text = d['query'] + ' [SEP] ' + nd
            texts.append(text)
        input_tokens = self.bert_tokenizer(texts, padding=True, truncation=True, return_tensors='pt').to('cuda')
        scores = self.bert_scorer(input_tokens)
        return scores

class DocLH_Logit(nn.Module):
    def __init__(self, model_path=None):
        super(DocLH_Logit,self).__init__()
        self.bert_scorer = BertLogitScorer()
        if model_path is not None:
            self.bert_scorer.load_state_dict(torch.load(model_path))
        self.bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    def forward(self, d):
        texts = []
        for pd in d['pos_doc']:
            text = d['query'] + ' [SEP] ' + pd
            texts.append(text)
        for nd in d['neg_doc']:
            text = d['query'] + ' [SEP] ' + nd
            texts.append(text)
        input_tokens = self.bert_tokenizer(texts, padding=True, truncation=True, return_tensors='pt').to('cuda')
        scores = self.bert_scorer(input_tokens)
        return scores
    

class QueryLH_CLS(nn.Module):
    def __init__(self, model_path=None):
        super(QueryLH_CLS,self).__init__()
        self.bert_scorer = BertClsFFN()
        if model_path is not None:
            self.bert_scorer.load_state_dict(torch.load(model_path))
        self.bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    
    def forward(self, d):
        texts = []
        for pd in d['pos_que']:
            text = d['doc'] + ' [SEP] ' + pd
            texts.append(text)
        for nd in d['neg_que']:
            text = d['doc'] + ' [SEP] ' + nd
            texts.append(text)
        input_tokens = self.bert_tokenizer(texts, padding=True, truncation=True, return_tensors='pt').to('cuda')
        scores = self.bert_scorer(input_tokens)
        return scores

class QueryLH_Logit(nn.Module):
    def __init__(self, model_path=None):
        super(QueryLH_Logit,self).__init__()
        self.bert_scorer = BertLogitScorer()
        if model_path is not None:
            self.bert_scorer.load_state_dict(torch.load(model_path))
        self.bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    
    def forward(self, d):
        texts = []
        for pd in d['pos_que']:
            text = d['doc'] + ' [SEP] ' + pd
            texts.append(text)
        for nd in d['neg_que']:
            text = d['doc'] + ' [SEP] ' + nd
            texts.append(text)
        input_tokens = self.bert_tokenizer(texts, padding=True, truncation=True, return_tensors='pt').to('cuda')
        scores = self.bert_scorer(input_tokens)
        return scores

In [12]:
model_Doc_log = DocLH_Logit(model_path = '/kaggle/input/wiki_doclh_logit/transformers/default/1/model_checkpoints/wiki_DocLH_Logits_epoch2.pth').to('cuda')
model_Doc_cls = DocLH_CLS(model_path = '/kaggle/input/wiki_doclh_cls/transformers/default/1/model_checkpoints/wiki_wiki_DocLH_CLS_epoch2.pth').to('cuda')
model_que_log = QueryLH_Logit(model_path = '/kaggle/input/query_cls_logits_wiki/transformers/default/1/model_checkpoints/wiki_QueryLH_Logits_epoch1.pth').to('cuda')
model_que_cls = QueryLH_CLS(model_path = '/kaggle/input/query_cls_logits_wiki/transformers/default/1/model_checkpoints/wiki_QueryLH_CLS_epoch1.pth').to('cuda')



config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please u

In [13]:
def prec(scores,k,gd):
    count = 0
    for i in range(k):
        count = count + scores[i][1]
    return float(count/k),float(min(k,gd)/k)

def mrr(scores):
    for i in range(len(scores)):
        if scores[i][1] == 1:
            return float(1.0/float(i+1))
        
def map_(scores,gd):
    sum = 0
    temp = 0
    for i in range(len(scores)):
        if scores[i][1] == 1:
            temp = temp + 1
            sum = sum + float((1.0*temp)/float(i+1))
            
    return float(sum/gd)
    
    

In [ ]:
model_names = ['DocLH_Logit','DocLH_CLS']
models = [model_Doc_log, model_Doc_cls]


In [15]:
from tqdm import tqdm
eval_scores = []
for mo in range(2):
    mo_name = model_names[mo]
    model = models[mo]
    prec1 = 0.0
    prec10 = 0.0
    max_prec1 = 0.0
    max_prec10 = 0.0
    mrr_= 0.0
    map__ = 0.0
    for test_d in tqdm(test_data, desc="Processing"):
        gold_d = []
        gold_d.extend([1] * len(test_d['pos_doc']))
        gold_d.extend([0] * len(test_d['neg_doc']))

        scores = model(test_d)
        scores = scores.tolist()
        my_pairs = [(scores[i],gold_d[i]) for i in range(len(scores))]
        my_pairs = sorted(my_pairs, key=lambda x: x[0], reverse=True)
        prec1 += prec(my_pairs,1,len(test_d['pos_doc']))[0]
        max_prec1 += prec(my_pairs,1,len(test_d['pos_doc']))[1]
        prec10 += prec(my_pairs,10,len(test_d['pos_doc']))[0]
        max_prec10 += prec(my_pairs,10,len(test_d['pos_doc']))[1]
        mrr_ += mrr(my_pairs)
        map__ += map_(my_pairs,len(test_d['pos_doc']))
        
    scores_data = {}
    scores_data['model_name'] = mo_name
    scores_data['prec1'] = float(prec1/len(test_data))
    scores_data['prec10'] =float(prec10/len(test_data))
    scores_data['mrr'] = float(mrr_/len(test_data))
    scores_data['map'] = float(map__/len(test_data))
    scores_data['max_prec1'] = float(max_prec1/len(test_data))
    scores_data['max_prec10'] = float(max_prec10/len(test_data))
    
    print(scores_data)
    
    eval_scores.append(scores_data)

    

In [ ]:
# print(eval_scores)

In [16]:
model_names = ['QueryLH_Logit','QueryLH_CLS']
models = [model_que_log,model_que_cls]

In [ ]:
from tqdm import tqdm
for mo in range(2):
    mo_name = model_names[mo]
    model = models[mo]
    prec1 = 0.0
    prec10 = 0.0
    max_prec1 = 0.0
    max_prec10 = 0.0
    mrr_= 0.0
    map__ = 0.0
    total = 0
    for test_d in tqdm(test_doc_data, desc="Processing"):
        try:
            gold_d = []
            gold_d.extend([1] * len(test_d['pos_que']))
            gold_d.extend([0] * len(test_d['neg_que']))

            scores = model(test_d)
            scores = scores.tolist()
            my_pairs = [(scores[i],gold_d[i]) for i in range(len(gold_d))]
            my_pairs = sorted(my_pairs, key=lambda x: x[0], reverse=True)
            prec1 += prec(my_pairs,1,len(test_d['pos_que']))[0]
            max_prec1 += prec(my_pairs,1,len(test_d['pos_que']))[1]
            prec10 += prec(my_pairs,10,len(test_d['pos_que']))[0]
            max_prec10 += prec(my_pairs,10,len(test_d['pos_que']))[1]
            mrr_ += mrr(my_pairs)
            map__ += map_(my_pairs,len(test_d['pos_que']))
            total+=1
        except:
            pass
        
        
    scores_data = {}
    scores_data['model_name'] = mo_name
    scores_data['prec1'] = float(prec1/total)
    scores_data['prec10'] =float(prec10/total)
    scores_data['mrr'] = float(mrr_/total)
    scores_data['map'] = float(map__/total)
    scores_data['max_prec1'] = float(max_prec1/total)
    scores_data['max_prec10'] = float(max_prec10/total)
    
    print(scores_data)
    
    eval_scores.append(scores_data)

    

In [ ]:
print(eval_scores)

In [ ]:
import json
with open('evaluate.json', 'w') as f:
    json.dump(eval_scores, f, indent=4)